In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.5 Pro > 멀티모달 모드 소개

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| 저자 |
| --- |
| [Eric Dong](https://github.com/gericdong) |
| [Holt Skinner](https://github.com/holtskinner) |
| 번역/편집 |
| [T Kim](https://github.com/aimldl) |

### 목표
Gemini 2.5 Pro 모델의 멀티모달 모드를 사용하는 방법을 배웁니다.

## 시작하기

### Python용 Google Gen AI SDK 설치

In [7]:
%pip install --upgrade --quiet google-genai

Note: you may need to restart the kernel to use updated packages.


### 노트북 환경 인증 (Colab 전용)
- Vertex AI Workbench 사용자는 생략
- Google Colab에서 이 노트북을 실행하는 경우, 아래 셀을 실행해서 환경을 인증하세요.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 라이브러리 가져오기

In [8]:
from IPython.display import HTML, Image, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    SafetySetting,
    ThinkingConfig,
    Tool,
    ToolCodeExecution,
)

### Vertex AI용 Google Cloud 프로젝트 사용
- Google Cloud 프로젝트에서 [Vertex AI API 활성화](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)합니다.
- Google Cloud 프로젝트의 Vertex AI Workbench Instance에서 실행할 경우, 자동으로 `PROJECT_ID`와 `LOCATION`을 불러옵니다.

In [9]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

print(f"PROJECT_ID={PROJECT_ID}")
print(f"LOCATION={LOCATION}")

PROJECT_ID=vertex-workbench-notebook
LOCATION=global


In [10]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Gemini 2.5 Pro 모델을 로딩
- [Gemini 2.5 Pro](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-pro?hl=ko)의 Model ID를 확인하려면 [여기](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-pro?hl=ko)를 클릭하세요. 

<img src="images/gemini2_5_pro_model_id.png">

In [11]:
MODEL_ID = "gemini-2.5-flash-preview-05-20"  # @param {type: "string"}

## 멀티모달 프롬프트 보내기
- Gemini 2.5 Pro 멀티모달 프롬프트를 지원하는 모델입니다.
- 다양한 소스에서 다음 데이터 유형 중 하나를 포함할 수 있습니다.

### 지원되는 데이터 유형

| 데이터 유형 | 소스 | MIME 유형 |
| :---------- | :------------------------------------ | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 텍스트 | 인라인, 로컬 파일, 일반 URL, Google Cloud Storage | `text/plain` `text/html` |
| 코드 | 인라인, 로컬 파일, 일반 URL, Google Cloud Storage | `text/plain` |
| 문서 | 로컬 파일, 일반 URL, Google Cloud Storage | `application/pdf` |
| 이미지 | 로컬 파일, 일반 URL, Google Cloud Storage | `image/jpeg` `image/png` `image/webp` |
| 비디오 | 로컬 파일, 일반 URL, Google Cloud Storage, YouTube | `video/mp4` `video/mpeg` `video/x-flv` `video/quicktime` `video/mpegps` `video/mpg` `video/webm` `video/wmv` `video/3gpp` |
| 오디오 | 로컬 파일, 일반 URL, Google Cloud Storage | `audio/aac` `audio/flac` `audio/mp3` `audio/m4a` `audio/mpeg` `audio/mpga` `audio/mp4` `audio/opus` `audio/pcm` `audio/wav` `audio/webm` |

멀티모달 사용 사례의 더 많은 예는 [이 노트북](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb)을 참조하세요.

### 로컬 이미지 사용하기
`images/example-image-night-shot.jpg`
<img src="images/example-image-night-shot.jpg">

In [12]:
with open("images/example-image-night-shot.jpg", "rb") as f:
    image = f.read()

prompt="이 사진에 대해 설명해죠. 사진에 있는 빌딩의 이름이 뭐야?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=image, mime_type="image/png"),
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

이 사진은 밤에 촬영된 매우 인상적인 도시 야경입니다. 사진 중앙에는 거대한 고층 빌딩이 우뚝 솟아 있으며, 특히 최상층 부분은 밝은 녹색 빛으로 화려하게 빛나고 있습니다. 빌딩의 창문들에서도 불빛이 새어 나와 전체적인 윤곽을 더욱 돋보이게 합니다.

빌딩 앞에는 키 큰 나무들이 줄지어 서 있어 도시와 자연의 조화로운 모습을 보여줍니다. 나무들도 아래에서 조명을 받아 잎사귀와 줄기가 밝게 빛나고 있습니다. 밤하늘은 짙은 남색으로, 빌딩의 화려한 조명과 아름다운 대비를 이룹니다. 전체적으로 웅장하고 현대적인 느낌을 주는 멋진 야경 사진입니다.

사진에 있는 빌딩의 이름은 **대한민국 서울에 위치한 '롯데월드타워 (Lotte World Tower)'**입니다. 한국에서 가장 높은 빌딩 중 하나로 유명하며, 밤에는 특별한 조명 연출로 시선을 사로잡습니다.

### YouTube URL에서 비디오 보내기
[YouTube 비디오](https://youtu.be/LxjW4_C3_Cc?si=RIt8xZud9ZulnicC)를 활용해서 비디오 이해
- 주의: 유튜브 링크는 visibility=Public이여야 합니다. 

In [13]:
prompt="최종 게임스코어는 몇 분쯤에 나오나요?"

video = Part.from_uri(
    file_uri="https://youtu.be/LxjW4_C3_Cc?si=RIt8xZud9ZulnicC",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

최종 게임 스코어는 **0분 37초**쯤에 나옵니다.

## 고객 데이터

prompt="최종 게임스코어는 몇 분쯤에 나오나요?"

video = Part.from_uri(
    file_uri="https://youtube.com/shorts/g-oQj4GKcT0",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

### 일반 URL에서 오디오 보내기
이 예는 [Kubernetes Podcast]](https://kubernetespodcast.com/) 에피소드의 오디오입니다.

오디오 파일을 실행하려면 다음 링크를 클릭하세요.
- [https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3](https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3)
- 크롬 브라우저의 새 탭이 열리고, 재생할 수 있습니다.

In [14]:
#prompt="Write a summary of this podcast episode."
prompt="이 팟캐스트 에피소드를 요약한 다음, 한국어로 번역해죠."
audio_file_uri="https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3"
mime_type="audio/mpeg"

In [15]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri=audio_file_uri,
            mime_type=mime_type,
        ),
        prompt,
    ],
    config=GenerateContentConfig(
        audio_timestamp=True,
        #thinking_config=thinking_config,
    ),
)

display(Markdown(response.text))

이 쿠버네티스 팟캐스트 프롬 구글의 KubeCon North America 2024 행사 취재 에피소드는 초기 뉴스 요약과 함께 참석자들과의 인터뷰를 다루고 있습니다.

**주요 뉴스:**
*   **프로젝트 졸업:** Cert-Manager와 Dapr이 CNCF를 졸업했습니다.
*   **Istio 업데이트:** Istio 버전 1.20이 출시되었으며, Ambient Mesh가 Z-tunnel 및 Waypoint 프록시의 안정화와 함께 GA(General Availability) 되었습니다.
*   **새로운 CNCF 이니셔티브:** 특허 트롤 퇴치를 돕기 위한 현상금 프로그램인 Cloud Native Heroes Challenge가 발표되었습니다.
*   **2025년 행사 일정:** CNCF는 다양한 지역에서 5개의 KubeCon + CloudNativeCon 행사, 1개의 Open Source SecurityCon, 전 세계적으로 30개의 Kubernetes Community Days를 포함한 2025년 행사 라인업을 공개했습니다.
*   **자격증:** Certified Backstage Associate, OpenTelemetry Certified Associate, Kyverno Certified Associate 등 3개의 새로운 클라우드 네이티브 자격증이 발표되었습니다. Linux Foundation은 또한 주요 Kubernetes 자격증(CKA, CKS, CKAD) 및 Linux Certified System Administrator 시험의 가격을 10% 인상한다고 발표했습니다.
*   **프로젝트 성장:** WasmCloud가 CNCF의 인큐베이팅 프로젝트로 합류했습니다. Spectro Cloud는 Kubernetes 관리 솔루션을 위해 시리즈 C 펀딩으로 7,500만 달러를 유치했습니다. Solo.io는 Gloo API Gateway를 CNCF에 기부했습니다.

**참석자 인터뷰:**
인터뷰에 참여한 참석자들은 KubeCon에서의 경험과 주요 시사점을 공유하며 여러 반복되는 주제를 강조했습니다:
*   **AI 및 머신러닝:** 클라우드 네이티브와 AI 통합 및 AI/ML 워크로드 스케줄링 최적화에 대한 상당한 관심이 있었습니다.
*   **보안:** Kubernetes 보안, GPU 모니터링, 취약점 처리, 클라우드 네이티브 환경에서 보안 관리의 복잡성에 대한 논의가 많았습니다. 많은 프로젝트가 보안 강화에 중점을 두고 있습니다.
*   **커뮤니티 및 연결:** (팬데믹 이후 특히) 대면 상호 작용, 동료 기여자들과의 만남, Contributor Summit(곧 Maintainer Summit으로 변경될 예정)과 같은 커뮤니티 행사 참여의 가치에 대한 강한 강조가 있었습니다.
*   **기술 심층 분석:** 참석자들은 Istio의 Ambient Mesh GA, Envoy 기반 애플리케이션 게이트웨이, Kubernetes 권한 부여의 현재 상태와 미래, 저지연/고성능 워크로드에 관심을 보였습니다.

---

**Korean Translation:**

This KubeCon North America 2024 coverage episode of the Kubernetes Podcast from Google features an initial news segment followed by interviews with attendees.

**News Highlights:**
*   **Project Graduations:** Cert-Manager and Dapr have graduated from the CNCF.
*   **Istio Update:** Istio version 1.20 was released, with Ambient Mesh becoming Generally Available (GA), including stable Z-tunnel and Waypoint Proxies.
*   **New CNCF Initiatives:** The Cloud Native Heroes Challenge was announced, a bounty program aimed at helping fight patent trolls.
*   **2025 Event Schedule:** The CNCF unveiled its 2025 event lineup, featuring five KubeCon + CloudNativeCon events across various regions, one Open Source SecurityCon, and 30 Kubernetes Community Days globally.
*   **Certifications:** Three new Cloud Native certifications were announced: Certified Backstage Associate, OpenTelemetry Certified Associate, and Kyverno Certified Associate. The Linux Foundation also announced a 10% price increase for its main Kubernetes certifications (CKA, CKS, CKAD) and the Linux Certified System Administrator exam.
*   **Project Growth:** WasmCloud joined the CNCF as an incubating project. Spectro Cloud raised $75 million in Series C funding for its Kubernetes management solutions. Solo.io donated its Gloo API Gateway to the CNCF.

**Attendee Interviews:**
Interviewees shared their experiences and key takeaways from KubeCon, highlighting several recurring themes:
*   **AI and Machine Learning:** A significant focus on integrating AI with cloud-native and optimizing AI/ML workload scheduling.
*   **Security:** This was a prevalent topic, with discussions on Kubernetes security, GPU monitoring, handling vulnerabilities, and the complexity of managing security in cloud-native environments. Many projects are focusing on increasing security.
*   **Community and Connection:** A strong emphasis on the value of face-to-face interactions, meeting fellow contributors (especially after the pandemic), and participating in community events like the Contributor Summit (soon to be Maintainer Summit).
*   **Technical Deep Dives:** Attendees were interested in Istio's Ambient Mesh GA, Envoy-based application gateways, the current state and future of Kubernetes authorization, and low-latency/high-performance workloads.